## Cross-Validation of YOLO Model for Coral Segmentation

This project utilizes the Ultralytics YOLOv8 library to train and evaluate a segmentation model for two coral classes: Madrepora oculata and Desmophyllum pertusum. K-Fold cross-validation is employed to ensure robust model training and evaluation.

### Requirements

Python: Make sure you have Python installed on your system.

Libraries: Install the necessary libraries with the following command:

Bash
pip install ultralytics
Usa el código con precaución.
content_copy

### Dataset:
You'll need a dataset with training images (train/images) and their corresponding segmentation labels (train/labels). Organize your data in the following structure:

├── train/

│   ├── images/

│   └── labels/

└── val/

    ├── images/
    
    └── labels/

### Usage

Open the file Cross-Validation.ipynb and replace 'runs/segment/train12/weights/best.pt' with the correct path to your trained YOLOv8 model.

### Code Explanation

Import libraries: The necessary libraries for script execution are imported.
Load model: The YOLOv8 model is loaded from the specified path.
Parameter definition: The number of folds (k) and the path to the data directory (data_dir) are set.
Cross-validation: A KFold object is created to split the data into k folds.
Iterate over folds:
Temporary YAML files are created with training and validation configuration for each fold.
The model is trained on the training set of the current fold.
The model is validated on the validation set of the current fold.
Validation results are saved.
The temporary YAML file is deleted.
Calculate average metrics: The average metrics of all folds are calculated and displayed.
Visualize results: The results of each fold and the average metrics are displayed in table format using pandas.

In [1]:
import os
from ultralytics import YOLO
from sklearn.model_selection import KFold

# Cargar el modelo
model = YOLO('runs/segment/train_vivo/weights/best.pt')  # Reemplaza con la ruta de tu modelo

# Rutas a tus carpetas de imágenes y etiquetas
data_dir = 'datasets/'  # Ruta a la carpeta principal del dataset

# Definir el número de pliegues
k = 5

# Crear objeto KFold
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Realizar la validación cruzada
results = []
for fold, (train_index, val_index) in enumerate(kf.split(os.listdir(os.path.join(data_dir, 'train/images')))):
    print(f'Fold {fold + 1}/{k}')

    # Crear archivos YAML temporales para cada pliegue
    with open(f'train_fold{fold}.yaml', 'w') as f:
        f.write(f'path: {data_dir}\n')
        f.write(f'train: train/images\n')
        f.write(f'val: val/images\n')
        f.write('nc: 2\n')  # Número de clases
        f.write("names: ['Madrepora oculata', 'Desmophyllum pertusum']\n")

    # Entrenar el modelo en el pliegue actual
    model.train(data=f'train_fold{fold}.yaml', epochs=10, batch=8)  # Ajusta los parámetros

    # Validar el modelo en el pliegue actual
    result = model.val(data=f'train_fold{fold}.yaml')  # Validar en el mismo pliegue
    results.append(result.results_dict)  # Guardar resultados en formato de diccionario

    # Eliminar el archivo YAML temporal
    os.remove(f'train_fold{fold}.yaml')

# Calcular métricas promedio
avg_metrics = {}
for metric_name in results[0].keys():
    metric_values = [x[metric_name] for x in results]
    avg_metrics[metric_name] = sum(metric_values) / len(metric_values)

print('Métricas promedio:', avg_metrics)

Fold 1/5
New https://pypi.org/project/ultralytics/8.2.23 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12036MiB)
engine/trainer: task=segment, mode=train, model=runs/segment/train_vivo/weights/best.pt, data=train_fold0.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /home/berto/Documents/iMagine/YOLO-seg/Train/datasets/train/labels.cache... 670 images, 100 backgrounds, 0 corrupt: 100%|██████████| 670/6
val: Scanning /home/berto/Documents/iMagine/YOLO-seg/Train/datasets/val/labels... 26 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:01<00:00

val: New cache created: /home/berto/Documents/iMagine/YOLO-seg/Train/datasets/val/labels.cache


Plotting labels to runs/segment/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 106 weight(decay=0.0), 117 weight(decay=0.0005), 116 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/train2
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/10       6.3G     0.6668      1.065     0.3857     0.8589         40        640: 100%|██████████| 84/84 [02:38<00:00,  1.89s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2

...


10 epochs completed in 0.110 hours.
Optimizer stripped from runs/segment/train23333/weights/last.pt, 92.3MB
Optimizer stripped from runs/segment/train23333/weights/best.pt, 92.3MB

Validating runs/segment/train23333/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12036MiB)
YOLOv8l-seg summary (fused): 295 layers, 45913430 parameters, 0 gradients, 220.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2


                   all         26        270       0.79      0.694      0.756      0.516      0.795        0.7      0.759      0.484
     Madrepora oculata         26        104       0.85      0.538      0.684      0.459       0.85      0.538      0.681      0.432
 Desmophyllum pertusum         26        166      0.729      0.849      0.829      0.574       0.74      0.861      0.837      0.536
Speed: 0.1ms preprocess, 13.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/segment/train23333
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12036MiB)
YOLOv8l-seg summary (fused): 295 layers, 45913430 parameters, 0 gradients, 220.1 GFLOPs


val: Scanning /home/berto/Documents/iMagine/YOLO-seg/Train/datasets/val/labels.cache... 26 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4


                   all         26        270       0.79      0.694      0.758      0.521      0.792      0.697      0.757       0.48
     Madrepora oculata         26        104       0.85      0.538      0.687      0.465       0.85      0.538      0.684      0.431
 Desmophyllum pertusum         26        166      0.729      0.849       0.83      0.577      0.734      0.855       0.83      0.528
Speed: 0.6ms preprocess, 24.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/segment/train233332
Métricas promedio: {'metrics/precision(B)': 0.7839099048122081, 'metrics/recall(B)': 0.7027815222590441, 'metrics/mAP50(B)': 0.7806390606884943, 'metrics/mAP50-95(B)': 0.5441976340976138, 'metrics/precision(M)': 0.7836344310604038, 'metrics/recall(M)': 0.6935423334114861, 'metrics/mAP50(M)': 0.7693902483967731, 'metrics/mAP50-95(M)': 0.5076188507304471, 'fitness': 1.1016377672537816}


In [48]:
import pandas as pd

# Crear DataFrame de Pandas con los resultados de cada pliegue
df_results = pd.DataFrame(results)

# Añadir una columna con el número de pliegue
df_results['Fold'] = range(1, k+1)

# Reordenar las columnas para que 'Fold' sea la primera
df_results = df_results[['Fold'] + [col for col in df_results.columns if col != 'Fold']]

print("Resultados por pliegue:")
display(df_results)  # Usar display para mostrar la tabla en el notebook

# Mostrar métricas promedio en formato de tabla
df_avg_metrics = pd.DataFrame([avg_metrics])
df_avg_metrics.index = ['Promedio']
print("\nMétricas promedio:")
display(df_avg_metrics)  # Usar display para mostrar la tabla en el notebook

Resultados por pliegue:


,Fold,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP50-95(M),fitness
0,1,0.733778,0.728742,0.774444,0.556892,0.730979,0.725730,0.763817,0.498658,1.103821
1,2,0.740343,0.690010,0.759502,0.525888,0.749299,0.697829,0.759357,0.482755,1.059664
2,3,0.695485,0.759384,0.781415,0.539901,0.785961,0.667123,0.769280,0.494145,1.085711
3,4,0.677701,0.754386,0.789031,0.546902,0.683034,0.756658,0.788192,0.493303,1.093907
4,5,0.739773,0.719184,0.763121,0.521131,0.759881,0.686909,0.753303,0.468490,1.042301



Métricas promedio:


,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP50-95(M),fitness
Promedio,0.717416,0.730341,0.773503,0.538143,0.741831,0.70685,0.76679,0.48747,1.077081


In [46]:
import pandas as pd

# Crear DataFrame de Pandas con los resultados de cada pliegue
df_results = pd.DataFrame(results)

# Añadir una columna con el número de pliegue
df_results['Fold'] = range(1, k+1)

# Reordenar las columnas para que 'Fold' sea la primera
df_results = df_results[['Fold'] + [col for col in df_results.columns if col != 'Fold']]

print("Resultados por pliegue:")
display(df_results)  # Usar display para mostrar la tabla en el notebook

# Mostrar métricas promedio en formato de tabla
df_avg_metrics = pd.DataFrame([avg_metrics])
df_avg_metrics.index = ['Promedio']
print("\nMétricas promedio:")
display(df_avg_metrics)  # Usar display para mostrar la tabla en el notebook

Resultados por pliegue:


,Fold,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP50-95(M),fitness
0,1,0.839217,0.657674,0.781185,0.566150,0.758901,0.706557,0.764427,0.516996,1.129393
1,2,0.740718,0.672524,0.751255,0.526230,0.740718,0.672524,0.748245,0.492120,1.066465
2,3,0.740305,0.737445,0.772860,0.544468,0.740305,0.737445,0.765884,0.514242,1.106713
3,4,0.775555,0.664446,0.748131,0.523529,0.772740,0.661434,0.740832,0.497034,1.067403
4,5,0.752505,0.690802,0.741473,0.519561,0.721567,0.703545,0.743318,0.492443,1.059282



Métricas promedio:


,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP50-95(M),fitness
Promedio,0.76966,0.684578,0.758981,0.535988,0.746846,0.696301,0.752541,0.502567,1.085851
